In [1]:
from operator import itemgetter
import itertools
import numpy as np
import pandas as pd
import imp
mt = imp.load_source('metrics', '../evaluation_metric/metrics.py')
from utils import merge_utils as mu
from utils import write_utils as wu
from utils import parse_utils as pu

In [2]:
def parse(sc, path):
    # Create rdd
    rdd = sc.textFile(path)
    # Select header to be removed
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
           .map(lambda x: x.split("\t"))
           .filter(lambda x: x[0] != '')
           .map(lambda x:(int(x[0]),setup_items(x[1])))
    )

    return rdd

def setup_items(items):
    item_rank = 0
    items_list = []
    items_split = items.split(',')
    
    for item in items_split:
        if item != '':
            item_rank += 1
            items_list.append((int(item),item_rank,0))
            
    return items_list

In [3]:
def generate(sc, rdd):
        
    new_rdd = (rdd.map(lambda x: ((x[0],x[1]),1))
                 .reduceByKey(lambda x,y:x+y)
                 .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                 .reduceByKey(lambda x,y:x+y)
                 .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                 )
        
    return new_rdd

In [4]:
test_users_rdd = sc.textFile("../../Dataset/target_users.csv")
header = test_users_rdd.first()
test_users_list = (test_users_rdd.filter(lambda x: x != header)
           .map(lambda x: x.split("\n"))
           .map(lambda x: int(x[0]))
           .collect()
    )

In [5]:
active_items = dict(pu.parseItemData(sc, "../../Dataset/items.csv")
               .filter(lambda x: x[1].active_during_test == 1)
               .map(lambda x: (x[0],1))
               .collect())

Interactions
--

In [11]:
interaction_rdd = (pu.parseInteractions(sc, "../../Dataset/interactions.csv")
                  .filter(lambda x: x.interactionType != 4)
                  .map(lambda x: (x.userId, x.itemId))
                  .filter(lambda x: x[0] in test_users_list)
                  .filter(lambda x: active_items.get(x[1]) != None))

In [12]:
interaction_rdd.take(1)

[(2512615, 1430344)]

In [13]:
int_rdd = generate(sc, interaction_rdd)

In [14]:
wu.prepareSubmission(int_rdd,"../../Submissions/ens_subs/interactions")

Submission file ../../Submissions/ens_subs/interactions created successfully


Impressions
---

In [6]:
impressions_rdd = (pu.parseImpressions(sc, "/Users/tommy/Downloads/impressions.csv")
                  .map(lambda x: (x.userId, x.itemIds))
                  .filter(lambda x: x[0] in test_users_list)
                  .flatMap(lambda r: [(r[0], x, 1) for x in r[1] if active_items.get(x) != None]))

In [7]:
impressions_rdd.take(1)

[(594937, 1469274, 1)]

In [8]:
imp_rdd = generate(sc, impressions_rdd)
print imp_rdd.take(1)

[(2134528, [(1984327, 313), (1386412, 267), (657183, 237), (1069281, 183), (1162250, 146), (1679143, 87), (1721694, 71), (1583705, 68), (1299461, 67), (704891, 58), (2446769, 57), (1570917, 53), (1729618, 52), (1140869, 42), (784737, 40), (1244196, 39), (266412, 37), (2489959, 26), (72465, 25), (1776330, 22), (468120, 8), (673299, 6)])]


In [9]:
wu.prepareSubmission(imp_rdd,"../../Submissions/ens_subs/impressions")

Submission file ../../Submissions/ens_subs/impressions created successfully
